In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import _pickle as pickle

# Selecting patients with full eligibility from age 2 to 5

In [2]:
pid_eli_map = {}
with open("/Users/xxz005/Desktop/Raw_Data/PFKJan09Jul17/Eligibility.txt") as infile:
    next(infile)
    for line in infile:
        line = line.split('|')
        pid = line[0]
        eli_month = line[1]
        if pid not in pid_eli_map:
            pid_eli_map[pid] = [eli_month]
        else:
            pid_eli_map[pid].append(eli_month)

In [3]:
pid_dob_map = {}
count = 0
with open("/Users/xxz005/Desktop/Raw_Data/PFKJan09Jul17/Claims.txt") as infile:
    next(infile)
    for line in infile:
        count+=1
        line=line.split("|")
        if len(line[51])<4:continue
        pid = line[0]
        dob=line[51][:9]
        pid_dob_map[pid] = dob

In [4]:
def eli_check(pid_eli_map, pid_dob_map):
    population = []
    for p in pid_dob_map:
        dob = datetime.strptime(pid_dob_map[p], '%d%b%Y')
        month_id = [datetime.strptime(x, '%Y%m') for x in pid_eli_map[p]]
        full_eli = [i for i in month_id if (i-dob).days >180 and (i-dob).days <1825]
        
        if len(set(full_eli))!=54:continue
        #print(full_eli)
        #if min(full_eli)>datetime.strptime('201001', '%Y%m'):continue
        #if min(full_eli)<datetime.strptime('200906', '%Y%m'):continue
        population.append(p)
    return population

In [5]:
population = eli_check(pid_eli_map, pid_dob_map)
len(population)

24223

In [6]:
selected_population = []

with open("/Users/xxz005/Desktop/Raw_Data/PFKJan09Jul17/Claims.txt") as infile:
    next(infile)
    for line in infile:
        line=line.split("|")
        pid = line[0]
        if pid not in population:continue
        selected_population.append(line)

# Build mapping tables: 1) from patient to claims. 2) from claim to information

In [15]:
pid_claim_map = {}
claim_info_map = {}

for line in selected_population:
    pid = line[0]
    claim_id = line[2]
    enc_id = line[141]
    gender = line[10]
    diag_set = line[13:16]+line[65:70]+line[93:110]
    paid = line[46]
    proc = line[21]
    majcat = line[39]
    dob = line[51][:9]
    svcdt = line[27][:9]
    if len(svcdt):svcdt = datetime.strptime(svcdt, '%d%b%Y')
    else:svcdt = datetime.strptime('08AUG8888', '%d%b%Y')
    
    if len(dob):dob = datetime.strptime(dob, '%d%b%Y')
    else:dob = datetime.strptime('08AUG8888', '%d%b%Y')
    
    pid_claim_id = pid+"_"+claim_id
    if pid not in pid_claim_map:
        pid_claim_map[pid] = [pid_claim_id]
    else:
        pid_claim_map[pid].append(pid_claim_id)
        pid_claim_map[pid] = list(set(pid_claim_map[pid]))
    
    if pid_claim_id not in claim_info_map:
        claim_info_map[pid_claim_id] = [[claim_id, diag_set, proc, majcat, svcdt, gender, dob, paid, enc_id]]
    else:
        claim_info_map[pid_claim_id].append([claim_id, diag_set, proc, majcat, svcdt, gender, dob, paid, enc_id])

In [16]:
pickle.dump(pid_claim_map, open('pid_claim_map','wb'))
pickle.dump(claim_info_map, open('claim_info_map','wb'))